# Welcome future python subsurface professionals! (yes, you!)


< Operational note: This works best in Firefox or Chrome, do NOT use edge or internet expolorer, netscape naviator, AOL, etc. >

Welcome to Transform 2020! This does not require an install (!!!) but it does require a google account. You can also run this in Anaconda, but I am not going to cover that in this course (and this notebook is not tested for that).

## Breaks
We will have two breaks at roughly 45 minutes in and 1.5 hours in. 

## Slack channel
If you have questions, please ask in the slack! I will not be monitoring the slack channel, but there will be a few pro's from SWUNG there to assist.

# Welcome to Colab!

Google Colab is a online, web hosted way to work through Jupyter Notebooks. They have a pro version that adds larger GPU's and more RAM for $10 USD a month. For today, the free version will work great. It can be accessed on any computer with an internet connection and a modern browser. 

#### Mounting Colab to Google Drive

The hardest part of Colab is getting the google drive mounted and finding your file locations. Google does this to keep your data 'safe' but does have some drawbacks. Because it is an ipython notebook, you can use the tricks of cd, ls, pwd, and others to navigate to your folder of interest.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

"ls" is the list command. 

In [0]:
ls

My folder name is t20-intro, you might need to change it to match your folder name.

In [0]:
cd gdrive/My Drive/t20-intro/

pwd is the command to display the path to your location

In [0]:
pwd

In [0]:
ls

 Unix cheet sheet is [here](https://files.fosswire.com/2007/08/fwunixref.pdf). I am not going to go into detail but it's good to know a few key ones!

# Python!

Finally! Let's get into why y'all are here. Python has some basic math functions built in:



In [0]:
2 + 2

In [0]:
3 * 11.1

In [0]:
print('hello!')

In [0]:
a = 5
b = 2.1

a * b 

In [0]:
# This is a code cell, but a # makes it a comment
2 * 2 + 3

python indexes 0,1,2,3 not 1,2,3,4 (like matlab)

In [0]:
c = 'rad'
c[0]

In [0]:
c[1]

There are whole classes devoted to basic python syntax, loops, functions, etc. This class is to just get you up and running with some key basics that can help with basic projects. Join us on the slack for some pointers!

## Importing and Using Packages

Packages are similar to Matlab toolboxes. You import them in a line of code in your notebook. 

In [0]:
import matplotlib.pyplot as plt

Ok, so we are telling the computer everytime we want to use the pyplot function from matplotlib, and we are going to call it plt as it's way shorter. Programmers are lazy. 

In [0]:
import pandas as pd
import numpy as np

Importing a few others that I use in (almost) every project. We can import them normally becuase Google pre-loaded them. Google does not pre-load everything that you need, sometimes you might need to install it.  ~*warning* ~ not every package can easily be imported in Colab. I have not figured out what makes some go and some not. But stats and plotting is usually pretty safe. Niche, one off, old, academic code, less so. 

In [0]:
!pip install lasio # the ! before the pip is important, also this is a shell command, not a python one

Just installed the lasio package using pip. Lasio is a .las file imput and output reader. It's a great way to read in a .las file (well logs, not LiDAR) to use in python. After installing it, you still need to import it. The github for lasio is [here](https://github.com/kinverarity1/lasio)

In [0]:
import lasio

There are hundreds of packages, not all of them work with Colab. We will install and import packages throughout the notebook.

### Wikipedia Data and Pandas

Let's import a csv (comma seperated variable plain text (ASCII) file) into Pandas. Pandas is an awesome way to slice and dice data. Data is stored in data frames (df) in the next cell. When you see examples online, usually df is the naming convention. But you can name it anything! Pandas has nothing to do with the nice animals, but everything to do with panelled data.



This table is from Wikipedia, from [here](https://en.wikipedia.org/wiki/List_of_geothermal_power_stations_in_the_United_States). I used https://wikitable2csv.ggor.de/ to make a csv from the table! Pretty nifty. 

In [0]:
df = pd.read_csv('GeothermalPowerPlants_wikipedia.csv')

Use the .head methodm to show the first few rows of the data frame. Normally you get 5 rows, but you can pass any number. In the next line I did 3.

In [0]:
df.head(3)

In [0]:
df.shape # Just to see how big it is

Congrats! We just made a pandas data frame. Let's dig in.

Let's drop the Ref column, and change the names of two of the columns so they are easier to call. We will also remove the commas from the dataframe. We are going to make a dictionary of the renamed columns first to define what we want to rename them to. 

In [0]:
col_dict = {"Capacity(MW)": "Capacity_MW", "Annual Generation(GWh)": "AG_GWh"} # dictionary

df = df.rename(columns=col_dict) # We pass this dictionary to the rename method. It wont change columns not in the dictionary.
df = df.drop(columns=['Ref'])
df.replace(',','', regex=True, inplace=True)
df.head()

In [0]:
df.Capacity_MW.values

Two of these are problems, with [note 2,3] after the numbers, and we just want the numbers! A hint about this is dtype=object, and not int64

In [0]:
# Split on the space, taking the values before the space
df.Capacity_MW = df.Capacity_MW.str.split(r" *", expand=True)[0] # splitting on the space delimiter 
df.Capacity_MW = pd.to_numeric(df.Capacity_MW) # Changing the type to numeric
df.Capacity_MW.values

New column, lets check out AG_GWh

In [0]:
df.AG_GWh.head() # the head(2) comment is just to show the first 2 

Same problem! Let's remove the years from this.

In [0]:
df.AG_GWh = df.AG_GWh.str.split(r" *", expand=True)[0]
df.AG_GWh = pd.to_numeric(df.AG_GWh)
df.AG_GWh.values

In [0]:
df.head()

Let's break up that location column. First step is to split it out, to only get the decimal degrees.

In [0]:
oldloc = df.Location[0]
oldloc

Wow, that is a gross string, unicode and all. Let's see how we can parse out the good stuff. 

In [0]:
df.LocationDD = df.Location.str.split(r" / ", expand=True)[1] # note the new column we are making
df.LocationDD.head()

Let's seperate decimal degrees into lat and long, and drop the degrees symbol, and N and W. Also convert it to a number from an object. 

In [0]:
# Splitting the decimal degree column, on the space and seperating out the lat's and long's

df.latDD_N  = df.LocationDD.str.split(r" *", expand=True)[0] 
df.longDD_W = df.LocationDD.str.split(r" *", expand=True)[1] #

df.latDD_N.replace('°N','', regex=True, inplace=True) # replacing the degree symbol with a space
df.longDD_W.replace('°W','', regex=True, inplace=True)

df.latDD_N.values # displaying the values

Whoa! Got some unicode going on, lets drop that as well, and convert to a number, and make west negative. 

In [0]:
df.latDD_N.replace('\ufeff','', regex=True, inplace=True) # doing more replacing
df.longDD_W.replace('\ufeff','', regex=True, inplace=True)

df['latDD_N'] = pd.to_numeric(df.latDD_N)
df['longDD_W'] = pd.to_numeric(df.longDD_W)

df.latDD_N = pd.to_numeric(df.latDD_N)
df.longDD_W = pd.to_numeric(df.longDD_W)

# Because it is in the west, easier to put a negative
df.longDD_W = df.longDD_W*-1 
df.longDD_W.values

Quick QC plot of the new lat and longs

In [0]:
plt.scatter(df.longDD_W.values, df.latDD_N)

Looks about right, the one in the far lower left is Big Island, Hawai'i, [Puna Geothermal Ventures](https://www.hawaiianelectric.com/clean-energy-hawaii/our-clean-energy-portfolio/renewable-energy-sources/geothermal/puna-geothermal-venture-(pgv))

In [0]:
df.head()

You can use groupby to pull in all values. Summing the lat and long (or years!) does not make any physical sense, but cool to see the capacity. 

In [0]:
df.groupby(['State']).sum()

Let's just display the columns that make sense:

In [0]:
df[['State','Capacity_MW','AG_GWh']].groupby(['State']).sum()

In [0]:
df.groupby(['Year']).sum()

If you want to know how many times each state was counted, you can call that directly:

In [0]:
df.Year.value_counts()

### QC plots with Matplotlib

Matplotlib is a great plotting package, and the gold standard for a lot of projects. They have a ton of examples [here](https://matplotlib.org/3.1.1/gallery/index.html). I won't be going into detail on how to use matplotlib, but would be happy to answer questions after the webinar. When I get stuck, I typically check out the example webpage and try to recreate it.

Matplotlib can get really verbose, and there are other packages that are better for interactive graphics. 

In [0]:
plt.figure(figsize=(8,6)) #calling the figure size
plt.bar(df.State, df.Capacity_MW) # Making a bar plot with state on the X axis, capacity on the Y
plt.xlabel('States', size=16) # Just a x axis label, size 16
plt.ylabel('Capacity in MW', size = 16)
plt.grid(True, axis='y') # turning on grid lines, only for the Y axis


# You can drag and drop the figure to save it, or uncomment the next line
# plt.savefig('figurename.png', dpi=300) 


Let's make a scatter plot with similar data

In [0]:
plt.figure(figsize=(8,6))
plt.scatter(df.Year.values, df.Capacity_MW.values, s=150, color='green', alpha=0.3 ) 
plt.grid(True)

plt.xlabel('Year', size=16)
plt.ylabel('Capacity in MW', size = 16)

This size is wrong on the bubble, play around with the plot to find the right size. 

In [0]:
plt.figure(figsize=(8,6))
plt.scatter(df.Year, df.Capacity_MW, s=500 ) # hint look here
plt.grid(True)
plt.yscale('log') # Play around with turning this on and off


plt.xlabel('Year', size=16)
plt.ylabel('Capacity in MW', size = 16)

If you think having a log axis works better with this data. Spend 5-10 minutes during the break changing some variables, and try to change the color in the scatter plot. Next code cell is supposed to be empty.

### Break #1!



# Well log input and output using lasio

lasio (.las input and output) is one of my all time favorite packages. It's updated often, and it works 99.5% of the time with .las files (well logs, not LiDAR data). The repo is [here](https://github.com/kinverarity1/lasio) on github. 

In [0]:
las = lasio.read('4900722147_722147B.las')

We just read in a las file! Let's do a quick quaility control, and make some plots.

In [0]:
dir(las)

So there is a lot of stuff going on with the las object we just created. Which is awesome! But can be daunting to get into what you are looking for.

In [0]:
las.curves

Wow, this well has a lot of curves! OK, we are going to make a data frame for the data in the .las file. This will behave really similar to the previous data frame

In [0]:
data = las.df() # This time we named the dataframe data and not df
data.head()

Scroll around! If you want to get a handle on stats of your well curves, using the describe function on a dataframe is awesome. 

In [0]:
data.to_csv('well_data.csv') # just saved it out!

Once this is in a data frame, you can export the curve data as a csv! This can be used in excel, spotfire, matlab or any other program you are more familiar with. This alone saves tons of time! While I prefer python, I know it's not for everyone.

In [0]:
data.describe()

If you want different percentiles in df.describe, you can pass those values

In [0]:
data.describe(percentiles=[0.1, 0.5, 0.9])

If you want to save out this table as a csv, you can do that in one line:

In [0]:
data.describe().to_csv("petro-stats.csv")

In [0]:
data.AHO10.min() # If you just want one of the stats

Let's use the quantile function to make some quick cutoffs (will be used in the future).

In [0]:
lowGR = data.GR.quantile(.20)
lowGR

In [0]:
highRES = data.AHO90.quantile(.95)
highRES

Feel free to make your own cutoffs here!

### Plotting up well log data

In [0]:
# Cross plot comparing GR to Deep Resistivity
plt.figure(figsize=(5,4), dpi=100) # figure size and dpi you can set here
plt.scatter(data.GR.values, data.AHO90.values, color='blue', marker='.', alpha=0.3)
plt.yscale('log') #log scale for Y axis

plt.grid(True)
plt.xlabel('GR', size=16)
plt.ylabel('Deep Resistivity', size=16)

In [0]:
# lets make a histogram of the GR Curve, with 20 bins, in green
plt.hist(data.GR.values, bins = 20, color='g', alpha = 0.5)
plt.xlabel('GR', size=16)
plt.ylabel('Count', size=16)

In [0]:
# line plot of the Gamma curve
plt.figure(figsize=(4,8), dpi=100)
plt.plot(data.GR.values, data.index, color='g')
plt.ylabel('Depth in Feet', size=16)
plt.ylim(2000,0)
plt.grid(True)


Let's add a depth track for deep resistivity, a title and a few other additional widigits. Also we will use the well name for the title:

In [0]:
las.header['Well'].WELL.value # we can pull the well name directly from the header

In [0]:
# line plot of the Gamma curve
plt.figure(figsize=(6,7), dpi=100)

plt.suptitle(las.header['Well'].WELL.value, size =16) # overall title

plt.subplot(121) # if we are going to make two plots, matplotlib calls it a subplot
plt.plot(data.GR.values, data.index, color='g') # the actual plot!
plt.ylabel('Depth in Feet', size=16)
plt.xlabel('API', size=16)
plt.ylim(2000,0) # the limit is reversed to go deep down
plt.grid(True) # Turning the grid on

plt.subplot(122)
plt.plot(data.AHO10.values, data.index, color='grey')
plt.plot(data.AHO90.values, data.index, color='black')
plt.xscale('log')
plt.grid(True)
plt.xlim(0.4,4000)
plt.ylim(2000,0)
plt.xlabel('OhmM', size=16)
plt.tick_params(labelleft=False)  

plt.savefig('awesome-plot.pdf', dpi=200)

Let's add a pay flag, remember the cutoffs we calcualted above? Let's use those. Using pythonic nomenclature, we will set a true flag, only when both conditions are met.

In [0]:
payMapper = (data.GR.values <= lowGR) & (data.AHO90.values >= highRES)
np.unique(payMapper)

If we want to see how many true and false statements there are:

In [0]:
np.bincount(payMapper) # your specific numbers might be a bit different

So the GR values have to be below the cutoff, and resistivity have to be above the cutoff.

In [0]:
# line plot of the Gamma curve
plt.figure(figsize=(5,8), dpi=100)

plt.subplot(131) # the subplot changed to allow for 3 columns
plt.plot(data.GR.values, data.index, color='g')
plt.ylabel('Depth in Feet', size=16)
plt.xlabel('API', size=16)
plt.ylim(2000,0)
plt.grid(True)

plt.subplot(132) # this is the new pay flag plot
plt.fill_between(payMapper, data.index, color='red') # used fill between, not plot
plt.ylim(2000,0)
plt.tick_params(labelleft=False)  
plt.xlabel('Pay Flag', size=16)
plt.grid(True)

plt.subplot(133)
plt.plot(data.AHO90.values, data.index, color='black')
plt.xscale('log')
plt.grid(True)
plt.xlim(0.4,4000)
plt.ylim(2000,0)
plt.xlabel('OhmM', size=16)
plt.tick_params(labelleft=False)  



Little heavy on the pay flag! But a good template for future work. Can be used for any sort of cutoff, or statistical analysis. Mess around with a plot below!

### Break #2

# Striplog

Striplog is an awesome package for  basic graphic logs, stratigraphic information, formations, and other geo stuff. The github is [here](https://github.com/agile-geoscience/striplog). This heavily borrows from [tutorials](https://github.com/agile-geoscience/striplog/tree/master/tutorial).

In [0]:
!pip install striplog

In [0]:
import striplog
striplog.__version__ #if this fails, just re run the above cells

### Lexicon

Striplog has a lot of geowords already preloaded. Think sand, shale, mudstone, salt, etc:

In [0]:
from striplog import Lexicon
print(Lexicon.__doc__)

In [0]:
lexicon = Lexicon.default()
lexicon #scroll around!

In [0]:
lexicon.synonyms

These abbrevations are common for mudlogs, there is a great turtorial on the github if you are interested in that

In [0]:
s = "grysh gn ss w/ sp gy sh"
lexicon.expand_abbreviations(s)

### Componet

In [0]:
from striplog import Component

In [0]:
print(Component.__doc__)

We define a new rock with a Python dict object:

In [0]:
r = {'colour': 'grey',
     'grainsize': 'vf-f',
     'lithology': 'sand'}
rock = Component(r)
rock

You can now call these componets!

In [0]:
rock['colour'] # who spelled this?! < Matt Hall/>

In [0]:
rock.summary()

In [0]:
rock.summary(fmt="My rock: {lithology} ({colour}, {grainsize!u})")



The formatting supports the usual s, r, and a:

    s: str
    r: repr
    a: ascii

Also some string functions:

    u: str.upper
    l: str.lower
    c: str.capitalize
    t: str.title

And some numerical ones, for arrays of numbers:

    + or ∑: np.sum
    m or µ: np.mean
    v: np.var
    d: np.std
    x: np.product



### Position

Positions define points in the earth, like a top, but with uncertainty. You can define:

    upper — the highest possible location
    middle — the most likely location
    lower — the lowest possible location
    units — the units of measurement
    x and y — the x and y location (these don't have uncertainty, sorry)
    meta — a Python dictionary containing anything you want

Positions don't have a 'way up'.

In [0]:
from striplog import Position
print(Position.__doc__)

In [0]:
params = {'upper': 95,
          'middle': 100,
          'lower': 110,
          'meta': {'kind': 'erosive', 'source': 'DOE'}
          }

p = Position(**params)
p

Even if you don't give a middle, you can always get z: the central, most likely position:

In [0]:
params = {'upper': 75, 'lower': 85}
p = Position(**params)
p

In [0]:
p.z

## Let's make a striplog!

I just introduced a bunch of boring things about dictionaries, etc. What makes this nice once you get it setup, is you can make some templates for common formations and lithotypes. We are going to do a simple one from this [github](https://github.com/ThomasMGeo/CSV2Striplog)

In [0]:
from striplog import Lexicon, Decor, Component, Legend, Interval, Striplog


### Make a legend

Most of the stuff in the dicts you made were about display — so they are going to make Decor objects. A collection of Decors makes a Legend. A Legend determines how a striplog is displayed.

First I'll make the components, since those are easy. I'll move 'train' into there too, since it is to do with the rocks, not the display. If it seems weird having 'train' in the Component (which is really supposed to be about direct descriptions of the rock, but the idea is that it's always the same for all specimens of that rock so it does fit here) then you could put it in data instead.


In [0]:
facies = {
    's': Component({'lithology': 'sandstone', 'train':'y'}),
    'i': Component({'lithology': 'interbedded', 'train':'y'}),
    'sh': Component({'lithology': 'shale', 'train':'y'}),
}

The next block of text could be less lines of code. The indenting is just a way to make it easier to read. Everyone has there own style of programming. We are just setting the decor of our future striplog plot. 

In [0]:
sandstone = Decor({
    'component': facies['s'],
    'colour': 'yellow',
    'hatch': '.',
    'width': '3',
})

interbedded = Decor({
    'component': facies['i'],
    'colour': 'darkseagreen',
    'hatch': '--',
    'width': '2',
})

shale = Decor({
    'component': facies['sh'],
    'colour': 'darkgray',
    'hatch': '-',
    'width': '1',
})


In [0]:
legend = Legend([sandstone, interbedded, shale])
legend

In [0]:
strip = Striplog.from_csv('t20-lith.csv')
strip[0]


##Deal with lithology¶

The lithology has been turned into a component, but it's using the abbreviation... I can't figure out an elegant way to deal with this so, for now, we'll just loop over the striplog and fix it. We read the data item's lithology ('s' in the top layer), then look up the correct lithology name in our abbreviation dictionary, then add the new component in the proper place. Finally, we delete the data we had.


In [0]:
strip[0].data

In [0]:
for s in strip:
    lith = s.data['lithology']
    s.components = [facies[lith]]
    s.data = {}
# Run once, not twice

In [0]:
strip[0]

In [0]:
strip.plot(legend)

Just plotted a simple strip log from a CSV! You can make striplogs 100 different ways, and I highly reccomend the turtorials on the github for more exploration. 

# Closing Thoughts


*   Start small, try to replicate current workflows
*   Know that in the beginning that stuff will take longer amounts of time
*   Try to work on a problem on your own for a little bit, but don't be afraid of asking for help on swung or stack overflow
*   Colab is great for quick analysis and some machine learning workflows, but doing a normal install of [Anaconda](https://www.anaconda.com/) on your own machine is a good idea for more advanced analysis or specific packages. 
*   Have fun! Sometimes it can be frustrating, but a quick walk or sleeping on problems can also help
*   Be excellent to each other

*fin!*